In [ ]:
!pip install stanza

In [18]:
import stanza

text = "har sagt"
doc = nlp(text)

print("{:C}".format(doc))

# text = har sagt
# sent_id = 0
1	har	ha	AUX	VB|PRS|AKT	Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act	2	aux	_	start_char=0|end_char=3|ner=O
2	sagt	säga	VERB	VB|SUP|AKT	VerbForm=Sup|Voice=Act	0	root	_	SpaceAfter=No|start_char=4|end_char=8|ner=O


In [8]:
phrases = [
    "sade",
    "har sagt",
    "akten",
    "den här akten"

]


['phrase_images/img_01.png',
 'phrase_images/img_02.png',
 'phrase_images/img_03.png',
 'phrase_images/img_04.png']